# Week 2: Identify Nearest Health Facilities

<span style="color:red">
**UPDATE**

Thank you for your analysis. Despite our warning efforts so far, the virus continues to spread rapidly. We want to get infected individuals treatment as quickly as possible, so we need your help to calculate which hospital or clinic is closest to each known infected individual in the population.
</span>

Your goal for this notebook will be to identify the nearest hospital or clinic for each infected person.

## Imports

In [1]:
import cudf
import cuml
import cupy as cp

## Load Population Data

Begin by loading the `lat`, `long` and `infected` columns from `'./data/week2.csv'` into a cuDF data frame called `gdf`.

In [95]:
gdf = cudf.read_csv('./data/week1.csv', usecols = ['lat', 'long', 'infected'])

## Load Hospital and Clinics Data

For this step, your goal is to create an `all_med` cuDF data frame that contains the latitudes and longitudes of all the hospitals (data found at `'./data/hospitals.csv'`) and clinics (data found at `'./data/clinics.csv'`).

In [40]:
all_med = cudf.read_csv('./data/hospitals.csv', usecols = ['Latitude', 'Longitude'])
am2 = cudf.read_csv('./data/clinics.csv', usecols = ['Latitude', 'Longitude'])
all_med.tail()
all_med.shape

(1229, 2)

In [54]:
am2.shape

(19082, 2)

In [62]:
all_med = cudf.concat([all_med, am2], ignore_index=True)
all_med.shape

(20308, 2)

Since we will be using the coordinates of those facilities, keep only those rows that are non-null in both  `Latitude` and `Longitude`.

In [71]:
#all_med = cudf.DataFrame(all_med)
all_med.dropna(subset=['Latitude', 'Longitude'], inplace=True)
#all_med.isnull().sum()
all_med.shape

(20301, 4)

## Make Grid Coordinates for Medical Facilities

Provided for you in the next cell (which you can expand by clicking on the "...", and contract again after executing by clicking on the blue left border of the cell) is the lat/long to grid coordinates converter you have used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `all_med` data frame you created in the last step.

In [64]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [78]:
cupy_lat = cp.asarray(all_med['Latitude'])
cupy_long = cp.asarray(all_med['Longitude'])

n_cupy_array, e_cupy_array = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
all_med['northing'] = cudf.Series(n_cupy_array).astype('float32')
all_med['easting'] = cudf.Series(e_cupy_array).astype('float32')
print(all_med.dtypes)
all_med.head()
all_med.isnull().sum()
all_med.dropna(subset=['northing', 'easting'], inplace=True)
all_med.shape

Latitude     float64
Longitude    float64
northing     float32
easting      float32
dtype: object


(20294, 4)

## Find Closest Hospital or Clinic for Infected

Fit `cuml.NearestNeighbors` with `all_med`'s `northing` and `easting` values, using the named argument `n_neighbors` set to `1`, and save the model as `knn`.

In [79]:
knn = cuml.NearestNeighbors(n_neighbors = 1)
med_locs = all_med[['northing', 'easting']]
knn.fit(med_locs)

NearestNeighbors()

Save every infected member in `gdf` into a new dataframe called `infected_gdf`.

In [105]:
infected_gdf = gdf[gdf['infected'] == True].reset_index()
gdf.shape
infected_gdf.head()

,index,lat,long,infected
0,28928759,54.472766,-1.654932,True
1,28930512,54.529717,-1.667143,True
2,28930904,54.512986,-1.589866,True
3,28932226,54.522322,-1.380694,True
4,28933748,54.541660,-1.613490,True


Create `northing` and `easting` values for `infected_gdf`.

In [106]:
cupy_lat = cp.asarray(infected_gdf['lat'])
cupy_long = cp.asarray(infected_gdf['long'])

n_cupy_array, e_cupy_array = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
infected_gdf['northing'] = cudf.Series(n_cupy_array).astype('float32')
infected_gdf['easting'] = cudf.Series(e_cupy_array).astype('float32')
print(infected_gdf.dtypes)
infected_gdf.isnull().sum()

index         int64
lat         float64
long        float64
infected       bool
northing    float32
easting     float32
dtype: object


index       0
lat         0
long        0
infected    0
northing    0
easting     0
dtype: int64

Use `knn.kneighbors` with `n_neighbors=1` on `infected_gdf`'s `northing` and `easting` values. Save the return values in `distances` and `indices`.

In [107]:
distances, indices = knn.kneighbors(infected_gdf[['easting', 'northing']], n_neighbors=1)

### Check Your Solution

`indices`, returned from your use of `knn.kneighbors` immediately above, should map person indices to their closest clinic/hospital indices:

In [108]:
indices.head()

0    18999
1    19995
2     1140
3    15158
4    18347
dtype: int64

Here you can print an infected individual's coordinates from `infected_gdf`:

In [109]:
infected_gdf.iloc[0] # get the coords of an infected individual (in this case, individual 0)

index       2.892876e+07
lat         5.447277e+01
long       -1.654932e+00
infected    1.000000e+00
northing    5.086701e+05
easting     4.223598e+05
Name: 0, dtype: float64

You should be able to used the mapped index for the nearest facility to see that indeed the nearest facility is at a nearby coordinate:

In [110]:
all_med.iloc[1234] # printing the entry for facility 1234 (replace with the index identified as closest to the individual)

Latitude         53.404556
Longitude        -2.831035
northing     390099.125000
easting      344757.531250
Name: 1234, dtype: float64

<div align="center"><h2>Please Restart the Kernel</h2></div>

...before moving to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)